In [1]:
import joblib
import os
import numpy as np
from tqdm import tqdm 
import torch
# from keras.backend.tensorflow_backend import set_session ## 和tf.keras不同

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}
#0 是默认值，输出所有信息
# 1 是屏蔽通知信息
# 2 是屏蔽通知和警告信息
###################################
###########loading dataset#########
###################################
l_N = 60   # lengthscale对的数量
k_ ='rbf'
nx = 32
ny = 32
num_samples = 100   # 每个随机场采样的样本数

# 构建高保真数据集

In [2]:
TrainDataFile = k_+"_l_N_"+str(l_N)+"_num_samples_"+str(num_samples)+\
                    "_nx_"+str(nx)+"_ny_"+str(ny)+"_train.pkl"
datadir = "/home/huangjg/MyFiles/deep-uq-paper/data"

TrainData = joblib.load(os.path.join(datadir,TrainDataFile))

In [7]:
from model import FNN

LFModel = FNN(1026,7,2)  # 低保真模型
save_Lmodel_name = k_+"_l_N_"+str(l_N)+"_num_samples_"+str(num_samples)+\
                    "_nx_"+str(nx)+"_ny_"+str(ny)+"_train.pth"
LFModel.load_state_dict(torch.load(os.path.join("/home/huangjg/MyFiles/deep-uq-paper/torch","pth",save_Lmodel_name)) ) #  保存模型参数)
LFModel.cuda()
LFModel.eval()

FNN(
  (seq_model): Sequential(
    (1): Linear(in_features=1026, out_features=421, bias=True)
    (1_Relu): ReLU()
    (2): Linear(in_features=421, out_features=173, bias=True)
    (2_Relu): ReLU()
    (3): Linear(in_features=173, out_features=71, bias=True)
    (3_Relu): ReLU()
    (4): Linear(in_features=71, out_features=30, bias=True)
    (4_Relu): ReLU()
    (5): Linear(in_features=30, out_features=12, bias=True)
    (5_Relu): ReLU()
    (6): Linear(in_features=12, out_features=5, bias=True)
    (6_Relu): ReLU()
    (7): Linear(in_features=5, out_features=3, bias=True)
    (8): Linear(in_features=3, out_features=600, bias=True)
    (8_Relu): ReLU()
    (9): Linear(in_features=600, out_features=1, bias=True)
  )
)

In [10]:
Inputs = TrainData["Inputs"]
Inputs = np.array(Inputs,dtype=np.float32)
Labels = TrainData["labels"]
num_data = Inputs.shape[0]
print(num_data)

6144000


In [20]:
i=0
batch_size = 2048
h_labels = Labels.copy()
while   (i+1)*batch_size<=num_data:
    inputs = Inputs[i*batch_size:(i+1)*batch_size]
    inputs = torch.from_numpy(inputs).cuda()
    pre_label = LFModel(inputs).detach().cpu().numpy()
    h_labels[i*batch_size:(i+1)*batch_size] = Labels[i*batch_size:(i+1)*batch_size]-pre_label.squeeze()
    i+=1
    


In [23]:
train_mean_label = np.mean(h_labels,axis=0)
train_std_label = np.std(h_labels,axis=0)
print(train_mean_label)
print(train_std_label)


0.0005513662228722254
0.03950530089254518


In [24]:
h_labels =(h_labels-train_mean_label) /train_std_label

HighTrainData = {
            "labels":h_labels,
            "normal":[train_mean_label,train_std_label,],}

HighTrainDataFile = k_+"_l_N_"+str(l_N)+"_num_samples_"+str(num_samples)+\
                    "_nx_"+str(nx)+"_ny_"+str(ny)+"_train_low.pkl"

joblib.dump(HighTrainData,filename=os.path.join(datadir,HighTrainDataFile))


['/home/huangjg/MyFiles/deep-uq-paper/data/rbf_l_N_60_num_samples_100_nx_32_ny_32_train_low.pkl']